In [ ]:
# Seleniume version 4+
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import pandas as pd
import os, glob, datetime, time, pytz

In [ ]:
# Use driver manager - uncomment below line
# service = Service(ChromeDriverManager().install())

# Use local driver
service = Service(executable_path= "YOUR_PATH")

# Create Driver Instance
driver = webdriver.Chrome(service=service)

# wait funtion
def loading_wait(driver, xpath, delay):
    time.sleep(1)
    try :
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH,xpath)))
    except :
        print('still wait')



driver.get('https://instructor.iclicker.com/#/courses/list/active')
# Please login

In [ ]:
delay = 180

df_class_history_ref = pd.DataFrame(columns=['date'])

# loop over sections
for i in list([0,1,2])[:] :
    driver.get('https://instructor.iclicker.com/#/courses/list/active')
    xpath = '//td[@class = "text-col ellipsis clickable"]'
    loading_wait(driver, xpath, delay)
    sections = [row.text for row in driver.find_elements(By.XPATH,xpath)]
    print(sections)
    section = sections[i]
    print(section) 
    # start
    driver.find_element(By.XPATH,'//button[contains(text(), "{sec}")]'.format(sec=section)).click()
    # 
    # get the class history for renaming
    time.sleep(3)
    # refreshing to avoid automated Chorme detection
    driver.get(driver.current_url)
    time.sleep(3)
    # 
    # load more
    while True : 
        try :
            driver.find_element(By.XPATH, '//button[@class = "btn loadMore"]').click()
            time.sleep(1)
        except :
            print ('done')
            break
    # 
    # load table 
    elm = driver.find_element(By.XPATH, "//*[@id='class-history']/div[2]/div/app-list/div/table")
    # BS soup reading
    soup = BeautifulSoup(elm.get_attribute('outerHTML'))
    # blank df
    df_history = pd.DataFrame()
    # 
    for idx, lecture in enumerate(soup.find_all('tr', class_='table-body-row')[:]) :
        i_date = lecture.findAll('span', class_='date')[0].get_text()
        for idx_tab, tabs in enumerate(lecture.findAll('table', class_='secondaryTable highlight-table')) :
            i_poll = tabs.findAll('button')[0].get_text().strip()
            i_attn = tabs.findAll('button')[1].get_text().strip()
            # print(idx, i_date, i_poll, i_attn)
            df_history = pd.concat([df_history, pd.DataFrame({'date':[i_date], 'poll_sec_{0}'.format(i+1):[i_poll]})])
    # 
    df_history['date'] = pd.to_datetime(df_history['date']).dt.strftime('%Y-%m-%d')
    df_history = df_history.sort_values(['date']).reset_index(drop=True)
    fname_hist = r'class_history {:%m-%d-%y} '.format(datetime.datetime.now()) + r'sec {0}.xlsx'.format(i+1)
    df_history.to_excel(fname_hist)
    # df_history
    
    # get the gradebook
    xpath = '//span[contains(text(), "Gradebook")]'
    loading_wait(driver, xpath, delay)
    driver.find_element(By.XPATH,xpath).click()

    xpath = '//button[contains(text(), "Export")]'
    loading_wait(driver, xpath, delay)
    driver.find_element(By.XPATH,xpath).click()

    xpath = '//input[@aria-label="Select All Sessions"]'
    loading_wait(driver, xpath, delay)
    driver.find_element(By.XPATH,xpath).click()

    xpath = '//button[@type = "submit" and contains(text(), "Export")]'
    loading_wait(driver, xpath, delay)
    driver.find_element(By.XPATH,xpath).click()
    time.sleep(2)

    xpath = '//button[@type = "button" and contains(text(), "Close")]'
    loading_wait(driver, xpath, delay)
    driver.find_element(By.XPATH,xpath).click()

    time.sleep(2)

    # modify file name and save it to onedrive.
    roster_down = sorted(glob.glob(os.path.expanduser("~")+'/Downloads/*.csv'), key=os.path.getatime, reverse=True)[0]
    roster_down

    if roster_down.endswith('iClicker_GradesExport_Reef_{:%m-%d-%y}.csv'.format(datetime.datetime.now(pytz.timezone('GMT')))) :
        print("file name matched")
        fname = 'Admin - iClicker/roster iClicker {:%m-%d-%y} '.format(datetime.datetime.now()) + 'Sec {0}.csv'.format(section[16:17])
        print(fname)
        try :
            os.remove(fname)
            os.rename(roster_down, fname)
        except :
            os.rename(roster_down, fname)
